In [30]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

In [31]:
import pandas as pd

In [32]:
from datasets import load_dataset
super_glue_dataset = load_dataset('super_glue', 'boolq')


/home/deon/miniconda3/envs/rakuten_project/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for super_glue contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/super_glue
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [33]:
glue=pd.DataFrame(super_glue_dataset['test'])
glue.head()

,question,passage,idx,label
0,is the first series 20 euro note still legal t...,20 euro note -- Until now there has been only ...,0,-1
1,do the champions league winners get automatic ...,2018–19 UEFA Champions League -- The final wil...,1,-1
2,can a bull snake kill a small dog,Bullsnake -- Bullsnakes are very powerful cons...,2,-1
3,are all nba playoff games best of 7,NBA playoffs -- All rounds are best-of-seven s...,3,-1
4,can i use my train ticket on the tram in manch...,Manchester station group -- The Manchester sta...,4,-1


In [34]:
"""import torch

# Check if a GPU is available and move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Tokenize input text
input_text = "Translate English to French: The house is wonderful."
inputs = tokenizer(input_text, return_tensors="pt")

# Move the input tensors to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate the output
outputs = model.generate(inputs['input_ids'], max_length=50)

# Decode the output
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(generated_text)
"""


'import torch\n\n# Check if a GPU is available and move the model to GPU\ndevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")\nmodel.to(device)\n\n# Tokenize input text\ninput_text = "Translate English to French: The house is wonderful."\ninputs = tokenizer(input_text, return_tensors="pt")\n\n# Move the input tensors to the same device as the model\ninputs = {key: value.to(device) for key, value in inputs.items()}\n\n# Generate the output\noutputs = model.generate(inputs[\'input_ids\'], max_length=50)\n\n# Decode the output\ngenerated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)\nprint(generated_text)\n'

In [35]:
#inputs.items()

In [36]:
def tokenize_function(dataset):
    return tokenizer(dataset['question'], dataset['passage'], truncation=True, padding='max_length')

tokenized_datasets = super_glue_dataset.map(tokenize_function, batched=True)


In [37]:
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['question', 'passage', 'idx', 'label', 'input_ids', 'attention_mask'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'passage', 'idx', 'label', 'input_ids', 'attention_mask'],
        num_rows: 3270
    })
    test: Dataset({
        features: ['question', 'passage', 'idx', 'label', 'input_ids', 'attention_mask'],
        num_rows: 3245
    })
})


In [38]:
import torch

device="cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

tokenized_datasets = tokenized_datasets.map(lambda x: {k: torch.tensor(v).to(device) for k, v in x.items() if k in ['input_ids', 'attention_mask']}, batched=True)


RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [10]:
super_glue_dataset

DatasetDict({
    train: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 9427
    })
    validation: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 3270
    })
    test: Dataset({
        features: ['question', 'passage', 'idx', 'label'],
        num_rows: 3245
    })
})

In [11]:
super_glue_dataset['test'][0]

{'question': 'is the first series 20 euro note still legal tender',
 'passage': '20 euro note -- Until now there has been only one complete series of euro notes; however a new series, similar to the current one, is being released. The European Central Bank will, in due time, announce when banknotes from the first series lose legal tender status.',
 'idx': 0,
 'label': -1}

In [12]:
from transformers import Trainer, TrainingArguments

#Define the training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy="epoch",     # evaluate each `epoch`
    per_device_eval_batch_size=16,   # batch size for evaluation
    # directory for storing logs
    logging_steps=10,
)

# Initialize the Trainer
trainer = Trainer(
    model=model,                     # the instantiated 🤗 Transformers model to be trained
    args=training_args,              # training arguments, defined above
    eval_dataset=tokenized_datasets['validation']  # evaluation dataset
)

# Evaluate the model
eval_results = trainer.evaluate()

print(eval_results)


/home/deon/miniconda3/envs/rakuten_project/lib/python3.11/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


[2024-09-27 12:20:09,930] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/deon/miniconda3/envs/rakuten_project/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/deon/miniconda3/envs/rakuten_project/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/deon/miniconda3/envs/rakuten_project/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/deon/miniconda3/envs/rakuten_project/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/deon/miniconda3/envs/rakuten_project/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/home/deon/miniconda3/envs/rakuten_project/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~r

ValueError: not enough values to unpack (expected 2, got 1)

In [20]:
outputs=model.generate(torch.tensor(tokenized_datasets['test'][0]['input_ids']).to(device),torch.tensor(tokenized_datasets['test'][0]['attention_mask']).to(device))

AttributeError: 'Tensor' object has no attribute 'update'